In [95]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

oapi_key=os.getenv("OPENAI_API_KEY")
api_key=os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)


In [96]:
from langchain_openai import ChatOpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

In [97]:
from langchain_community.document_loaders import PyPDFLoader

namespace = "resume2"
data_applicant = (
    {
        "name": "sarah",
        "file_path": "./dummy-cv/sarah-highly_accurate.pdf",
        "gender": "female",
    },
    {
        "name": "michael",
        "file_path": "./dummy-cv/michael-highly_accurate.pdf",
        "gender": "male",
    },
    {
        "name": "emily",
        "file_path": "./dummy-cv/emily-somewhat-relevant.pdf",
        "gender": "female",
    },
    {
        "name": "david",
        "file_path": "./dummy-cv/david-less-relevant.pdf",
        "gender": "male",
    },
    {
        "name": "anna",
        "file_path": "./dummy-cv/anna-irelevant.pdf",
        "gender": "female"
    },
    {
        "name": "fathur",
        "file_path": "./dummy-cv/sample-cv.pdf",
        "gender": "male"
    }
)

In [98]:
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=oapi_key,
)

example_question = {
    "1": "What is your name?",
    "2": "What is your email address?",
    "3": "What is your phone number?",
    "4": "Do you have any work experience in TypeScript development? If yes, please describe.",
    "5": "How long have you been working with TypeScript?",
    "6": "What is your experience with TypeScript frameworks like NestJS or Next.js?",
    "7": "Have you ever worked with type definitions (`.d.ts` files) in TypeScript? If yes, can you provide an example?",
    "8": "Do you have experience with JavaScript/TypeScript build tools like Webpack or Vite?",
    "9": "Do you have any experience integrating TypeScript with backend technologies such as Node.js?",
    "10": "Have you worked with TypeScript in a large-scale project? If so, what was your role and what challenges did you face?"
}

In [99]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Read the following text and answer the question, if you don't know the answer, just fill null. Returning a JSON object with an key for each question and the answer as the value. text: {text}.question: {question}""")
json_parser = SimpleJsonOutputParser()

json_chain = prompt | llm | json_parser

In [100]:
prompt_score = PromptTemplate.from_template(
    """
        You are provided with a job description, a text (which includes a question and the candidate's answer), and your task is to evaluate the relevance of the candidate's response based on the job description. Your goal is to return a JSON object that contains:
        'summary': A brief summary of the candidate's answer.
        'reason': Justification for the relevance score based on the job description.
        'score': A relevance score for the candidate’s response to the job description. The scoring scale is:
            0-20: Not relevant
            21-40: Less relevant
            41-80: Relevant
            81-100: Perfectly relevant
        
        job_description: {job_description}
        text: {text}
        question: {question}
    """
)

score_chain = prompt_score | llm | json_parser

py_desc= 'Job Title: Python Developer. Overview: We are looking for a Python Developer to join our team to design, develop, and maintain scalable applications and APIs. You will collaborate closely with other developers, product managers, and stakeholders to ensure high-quality deliverables. Responsibilities: Develop and maintain Python-based applications, build and maintain RESTful APIs, debug and troubleshoot software bugs, optimize performance, write clean and maintainable code, collaborate on software architecture, conduct code reviews, and assist in writing test cases. Qualifications: Strong proficiency in Python and familiarity with its libraries and frameworks (e.g., Django, Flask), understanding of RESTful services and API integrations, experience with version control systems like Git, excellent problem-solving and debugging skills, and experience working in an Agile environment. Nice-to-Have: Experience with asynchronous programming (e.g., asyncio, Celery), familiarity with cloud services like AWS, Google Cloud, or Azure, experience with CI/CD pipelines, and proficiency with testing tools such as Pytest or Unittest. How to Apply: Submit your application through our website or via [Application Link]; only shortlisted candidates will be contacted.'
job_description = "Job Title: TypeScript Developer. Overview: We are looking for a TypeScript Developer to join our team. You will work on building and maintaining web applications, collaborating with other developers to deliver high-quality software. This is a great opportunity to work in a dynamic environment and grow your skills. Responsibilities: Develop and maintain applications using TypeScript. Build and maintain APIs. Troubleshoot and fix software bugs. Collaborate with team members to deliver projects. Participate in code reviews and testing. Qualifications: Experience with TypeScript and JavaScript. Familiarity with modern front-end frameworks like React, Angular, or Vue. Understanding of RESTful APIs. Experience with version control (Git). Good problem-solving skills. Nice-to-Have: Experience with Node.js or backend development. Familiarity with cloud services (AWS, Azure). Experience with testing tools like Jest. How to Apply: Submit your application through our website or via [Application Link]. Only selected candidates will be contacted."
sr_desc='Job Title: Senior TypeScript Developer. Overview: We are looking for a Senior TypeScript Developer to lead the development of high-quality web applications and mentor the development team. Responsibilities: Design and build scalable applications using TypeScript, lead architectural decisions, oversee code quality, guide best practices, build and maintain RESTful APIs, debug complex issues, and conduct code reviews. Qualifications: Extensive experience with TypeScript and modern front-end frameworks (React, Angular, Vue), deep understanding of RESTful APIs, strong expertise in version control (Git), and proven leadership skills. Nice-to-Have: Experience with Node.js, cloud services (AWS, Azure), and advanced testing tools (Jest, Cypress). How to Apply: Apply through [Application Link]; only selected candidates will be contacted.'
kt_desc="Job Title: Kotlin Developer. Overview: We are seeking a Kotlin Developer to design, develop, and maintain robust applications and services. Responsibilities: Build and maintain applications using Kotlin, develop and integrate APIs, troubleshoot and resolve issues, contribute to code reviews, and collaborate on software design and architecture. Qualifications: Strong experience with Kotlin and Android development, knowledge of modern software development practices, experience with RESTful APIs, and proficiency in version control (Git). Nice-to-Have: Experience with Spring Boot, cloud platforms (AWS, GCP, Azure), and testing frameworks (JUnit, Espresso). How to Apply: Submit your application via [Application Link]; only shortlisted candidates will be contacted."


In [101]:
from langchain_core.documents import Document

index_name = "applicants-2"

openai_embedding_small_3_dimentions = 1536

pc.create_index(
    name=index_name,
    dimension=openai_embedding_small_3_dimentions,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

for x in data_applicant:
    loader = PyPDFLoader(x["file_path"], extract_images=True)
    pages = loader.load()

    doc: list[Document]= []

    result = json_chain.invoke({
        "text": list(map(lambda x: x.page_content, pages)),
        "question": example_question
    })

    result_scoring = score_chain.invoke({
        "text": list(map(lambda x: x.page_content, pages)),
        "question": result,
        "job_description": job_description
    })

    for y in pages:
        doc.append(
            Document(y.page_content, metadata={ "gender": x["gender"], "name": x["name"], "score": result_scoring["score"], "reason": result_scoring["reason"], "summary": result_scoring["summary"]})
        )

    docsearch = PineconeVectorStore.from_documents(
        embedding=embedding,
        index_name=index_name,
        documents=doc,
        namespace=namespace
    )


In [102]:
# import click

# def pprompt(question: str, default: str) -> None:
#     res = click.prompt(text=f"{question}\n", default=value if value != "null" else "" ,type=str)
#     if res == "":
#        return pprompt(question, default)
    
#     return res

# for key, value in result.items():
#     res: str = pprompt(example_question[key], default=value)
#     result[key] = res



In [108]:
query = "i need candidate that have experience on typescript but Prefer for fullstack engineer that can lead small teams"

vectors = embedding.embed_query(query)
index = pc.Index(name=index_name)

qr = index.query(
    namespace=namespace, 
    top_k=5,
    filter={
        "score": {"$gt": 30},
    },
    vector=vectors,
    include_values=False,
    include_metadata=True
)


In [109]:
qr

{'matches': [{'id': '5cf01024-1fac-4192-9ba9-08468a38b040',
              'metadata': {'gender': 'female',
                           'name': 'sarah',
                           'reason': "Sarah's extensive experience in "
                                     'TypeScript development directly aligns '
                                     'with the job requirements. Her '
                                     'leadership role and experience in '
                                     'building applications using modern '
                                     'frameworks like React, as well as her '
                                     'familiarity with RESTful APIs and '
                                     'cloud-based solutions, demonstrate her '
                                     'capability to fulfill the '
                                     'responsibilities outlined in the job '
                                     'description.',
                           'score': 100.0,
        